In [5]:
%matplotlib inline
from __future__ import print_function, division
path = "data/baidu/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 760 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [3]:
tokens = ['0','1','2', '3','4','5','6','7','8','9','(',')', '+', '-', '*', 'N']

# 准备数据

### 加载图片

In [4]:
gen = image.ImageDataGenerator()
batches = gen.flow_from_directory(path+'/image_contest_level_1', 
                                  batch_size=1, 
                                  target_size=(30,90),
                                  shuffle=False,
                                  class_mode=None,
                                  color_mode='grayscale'
                                 )


Found 100000 images belonging to 1 classes.


In [5]:
#imgs = np.concatenate([batches.next() for i in range(batches.nb_sample)])

imgs = load_array(path+'imgs0.dat')   #一般直接加载保存好的数据，快速加载

### 处理标签

In [6]:
x2label_idx = [int(f[8:][:-4]) for f in batches.filenames]
label_texts = []
lines = []
with open(path+'labels.txt') as f:
    lines = f.readlines()
label_texts = [l.strip().split()[0].ljust(7, 'N') for l in lines]
token2idx={char:i for i, char in enumerate(tokens)}
labels = np.array([[token2idx[token] for token in ts] for ts in label_texts])

In [7]:
idxs = np.random.permutation(imgs.shape[0])
mask = np.random.rand(len(imgs)) < 0.8
trn_idxs = idxs[mask]
val_idxs = idxs[~mask]

x_trn = imgs[trn_idxs]
y_trn_m = labels[np.array(x2label_idx)[trn_idxs]]
y_trn = np.expand_dims(np.stack(y_trn_m, axis=0), -1)

x_val = imgs[val_idxs]
y_val_m = labels[np.array(x2label_idx)[val_idxs]]
y_val = np.expand_dims(np.stack(y_val_m, axis=0), -1)

# 训练

In [8]:
def get_model():
    return Sequential([
            BatchNormalization(axis=1, input_shape=(1, 30, 90)),
        
            Convolution2D(16,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D(),
        
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            Convolution2D(128,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D(),

            Flatten(),
            BatchNormalization(),

            Dense(98, activation='relu'),
            BatchNormalization(),

            Dense(98, activation='relu'),
            BatchNormalization(),
        
            Reshape((7, 14)),
            TimeDistributed(Dense(16, activation='softmax'))
        ])

### 8-2划分测试集 验证集

In [9]:
#这里做数据增强 
da = image.ImageDataGenerator(zoom_range=0.05, shear_range=0.05, channel_shift_range=20,
                             rotation_range=5, height_shift_range=0.03, width_shift_range=0.03)
trn_batches = da.flow(x_trn, y_trn, batch_size=64, shuffle=True)

model = get_model()
model.compile(Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(trn_batches, len(x_trn), nb_epoch=1, validation_data=(x_val, y_val))

model.optimizer.lr=0.1
model.fit_generator(trn_batches, len(x_trn), nb_epoch=5, validation_data=(x_val, y_val))

model.optimizer.lr=0.05
model.fit_generator(trn_batches, len(x_trn), nb_epoch=10, validation_data=(x_val, y_val))

model.optimizer.lr=0.0125
model.fit_generator(trn_batches, len(x_trn), nb_epoch=10, validation_data=(x_val, y_val))

model.optimizer.lr=0.0015625
model.fit_generator(trn_batches, len(x_trn), nb_epoch=10, validation_data=(x_val, y_val))

Epoch 1/1
80187/80187 [==============================] - 90s - loss: 0.4456 - acc: 0.8793 - val_loss: 0.0273 - val_acc: 0.9945
Epoch 1/5
80187/80187 [==============================] - 90s - loss: 0.0472 - acc: 0.9875 - val_loss: 0.0101 - val_acc: 0.9976
Epoch 2/5
80187/80187 [==============================] - 90s - loss: 0.0297 - acc: 0.9916 - val_loss: 0.0057 - val_acc: 0.9985
Epoch 3/5
80187/80187 [==============================] - 90s - loss: 0.0234 - acc: 0.9932 - val_loss: 0.0051 - val_acc: 0.9987
Epoch 4/5
80187/80187 [==============================] - 90s - loss: 0.0219 - acc: 0.9936 - val_loss: 0.0036 - val_acc: 0.9991
Epoch 5/5
80187/80187 [==============================] - 91s - loss: 0.0175 - acc: 0.9948 - val_loss: 0.0036 - val_acc: 0.9991
Epoch 1/5
80187/80187 [==============================] - 91s - loss: 0.0147 - acc: 0.9957 - val_loss: 0.0021 - val_acc: 0.9994
Epoch 2/5
80187/80187 [==============================] - 91s - loss: 0.0128 - acc: 0.9963 - val_loss: 0.0060 - 

In [15]:
model.optimizer.lr=0.003125
model.fit_generator(trn_batches, len(x_trn), nb_epoch=5, validation_data=(x_val, y_val))

Epoch 1/5
80187/80187 [==============================] - 92s - loss: 0.0042 - acc: 0.9987 - val_loss: 6.4543e-04 - val_acc: 0.9998
Epoch 2/5
80187/80187 [==============================] - 93s - loss: 0.0046 - acc: 0.9986 - val_loss: 9.1711e-04 - val_acc: 0.9997
Epoch 3/5
80187/80187 [==============================] - 92s - loss: 0.0053 - acc: 0.9985 - val_loss: 7.2364e-04 - val_acc: 0.9998
Epoch 4/5
80187/80187 [==============================] - 92s - loss: 0.0050 - acc: 0.9986 - val_loss: 6.3280e-04 - val_acc: 0.9998
Epoch 5/5
80187/80187 [==============================] - 92s - loss: 0.0042 - acc: 0.9988 - val_loss: 7.6617e-04 - val_acc: 0.9998


In [17]:
model.optimizer.lr=0.001
model.fit_generator(trn_batches, len(x_trn), nb_epoch=10, validation_data=(x_val, y_val))

Epoch 1/10
80187/80187 [==============================] - 90s - loss: 0.0039 - acc: 0.9989 - val_loss: 8.0066e-04 - val_acc: 0.9997
Epoch 2/10
80187/80187 [==============================] - 90s - loss: 0.0045 - acc: 0.9987 - val_loss: 0.0025 - val_acc: 0.9991
Epoch 3/10
80187/80187 [==============================] - 90s - loss: 0.0044 - acc: 0.9987 - val_loss: 5.1622e-04 - val_acc: 0.9998
Epoch 4/10
80187/80187 [==============================] - 91s - loss: 0.0041 - acc: 0.9987 - val_loss: 5.3912e-04 - val_acc: 0.9998
Epoch 5/10
80187/80187 [==============================] - 91s - loss: 0.0036 - acc: 0.9989 - val_loss: 0.0016 - val_acc: 0.9995
Epoch 6/10
80187/80187 [==============================] - 91s - loss: 0.0038 - acc: 0.9990 - val_loss: 7.1333e-04 - val_acc: 0.9998
Epoch 7/10
80187/80187 [==============================] - 92s - loss: 0.0035 - acc: 0.9990 - val_loss: 2.6773e-04 - val_acc: 0.9999
Epoch 8/10
80187/80187 [==============================] - 92s - loss: 0.0039 - acc: 

In [20]:
model.optimizer.lr=0.0001
model.fit_generator(trn_batches, len(x_trn), nb_epoch=5, validation_data=(x_val, y_val))

Epoch 1/5
80187/80187 [==============================] - 90s - loss: 0.0033 - acc: 0.9991 - val_loss: 4.9209e-04 - val_acc: 0.9998
Epoch 2/5
80187/80187 [==============================] - 90s - loss: 0.0033 - acc: 0.9990 - val_loss: 2.5661e-04 - val_acc: 0.9999
Epoch 3/5
80187/80187 [==============================] - 91s - loss: 0.0035 - acc: 0.9990 - val_loss: 5.8768e-04 - val_acc: 0.9998
Epoch 4/5
80187/80187 [==============================] - 91s - loss: 0.0032 - acc: 0.9991 - val_loss: 3.8933e-04 - val_acc: 0.9999
Epoch 5/5
80187/80187 [==============================] - 92s - loss: 0.0030 - acc: 0.9991 - val_loss: 2.9430e-04 - val_acc: 0.9999


In [21]:
model.optimizer.lr=0.00005
model.fit_generator(trn_batches, len(x_trn), nb_epoch=5, validation_data=(x_val, y_val))

Epoch 1/5
80187/80187 [==============================] - 90s - loss: 0.0033 - acc: 0.9990 - val_loss: 4.4056e-04 - val_acc: 0.9999
Epoch 2/5
80187/80187 [==============================] - 91s - loss: 0.0037 - acc: 0.9989 - val_loss: 4.4841e-04 - val_acc: 0.9999
Epoch 3/5
80187/80187 [==============================] - 92s - loss: 0.0029 - acc: 0.9992 - val_loss: 3.9159e-04 - val_acc: 0.9999
Epoch 4/5
80187/80187 [==============================] - 92s - loss: 0.0035 - acc: 0.9990 - val_loss: 3.9782e-04 - val_acc: 0.9999
Epoch 5/5
80187/80187 [==============================] - 92s - loss: 0.0035 - acc: 0.9990 - val_loss: 3.4561e-04 - val_acc: 0.9999


In [3]:
import numpy as np

np.zeros([12, 1]) + 20

array([[ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.],
       [ 20.]])

Object `K.ctc_batch_cost` not found.


In [6]:
??K.ctc_batch_cost